<a href="https://colab.research.google.com/github/j-jayes/interlude-one/blob/main/Geocoding_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geocoding and Visualizing Place Names

In this notebook, we'll be using the Google Maps Geocoding API to convert place names from a CSV file into geographic coordinates. We will then visualize these locations on a map using Folium and save the geocoded results to a new CSV file.

You can read about getting an API key [here](https://developers.google.com/maps/documentation/geocoding/get-api-key).

## Import Necessary Libraries


In [ ]:
# Import necessary libraries
import requests  # To make API requests
import pandas as pd  # To handle CSV files and dataframes
import folium  # To visualize data on a map
from folium import Marker


## Load the Data

Let's start by loading the data from the CSV file.


In [ ]:
# Load the CSV file into a DataFrame
filename = 'https://raw.githubusercontent.com/j-jayes/interlude-one/main/posts/2023-10-02-geocoding/data/places_to_geocode.csv'  # Replace with your CSV file containing place names.
df = pd.read_csv(filename, delimiter=';')

# Display the first few rows of the dataframe
df.head()


,place_name,arbitrary_value
0,Stockholm,35
1,Gothenburg,12
2,Malmö,27
3,Uppsala,19
4,Västerås,22


## Geocoding

Now, let's use the Google Maps Geocoding API to get the latitude and longitude for each place name in our DataFrame.


In [ ]:
# Define the geocoding function
def geocode(address, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address, 'key': api_key}
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        return lat, lng
    else:
        return None, None

# Add columns for latitude and longitude
df['Latitude'] = None
df['Longitude'] = None

# Google Maps Geocoding API key
api_key = 'YOUR_GOOGLE_API_KEY'  # Replace with your Google API key

# Loop over the rows and geocode each address
for idx, row in df.iterrows():
    address = row['place_name']  # Replace 'place_name' with the name of the column containing the place names.
    lat, lng = geocode(address, api_key)
    df.loc[idx, 'Latitude'] = lat
    df.loc[idx, 'Longitude'] = lng

# Display the first few rows of the updated dataframe
df.head()


,place_name,arbitrary_value,Latitude,Longitude
0,Stockholm,35,59.329323,18.068581
1,Gothenburg,12,57.70887,11.97456
2,Malmö,27,55.604981,13.003822
3,Uppsala,19,59.858564,17.638927
4,Västerås,22,59.6099,16.544809


## Visualizing Geocoded Data

Let's visualize the geocoded data on a map using Folium. The map will automatically zoom to the bounds of the geocoded locations, and each marker's popup will display the place name and the arbitrary value associated with that location.



In [ ]:
# Create a base map; initially centered on a default location
m = folium.Map(location=[20,0], zoom_start=2)

# List to hold the lat-long values, which will be used to fit the map to bounds of the data.
lat_lng_list = []

# Add markers to the map for each geocoded location
for idx, row in df.iterrows():
    if row['Latitude'] and row['Longitude']:
        # Appending the latitude, longitude values to lat_lng_list
        lat_lng_list.append([row['Latitude'], row['Longitude']])

        # Creating a popup string containing place name and arbitrary value
        popup_str = f"{row['place_name']}<br>Arbitrary Value: {row['arbitrary_value']}"

        # Adding a marker to the map with the popup
        Marker([row['Latitude'], row['Longitude']], popup=popup_str).add_to(m)

# Fitting the map to the bounds of the geocoded locations
if lat_lng_list:
    m.fit_bounds(lat_lng_list)

# Display the map
m


## Save Geocoded Data

Finally, let's save our geocoded data to a new CSV file.


In [ ]:
# Save the DataFrame with the geocoded data to a new CSV file
output_filename = 'geocoded_locations.csv'  # Replace with your desired output filename.
df.to_csv(output_filename, index=False)

print(f'The geocoded data has been saved to {output_filename}')


The geocoded data has been saved to geocoded_locations.csv
